# TSPTW

In the traveling salesperson problem with time windows (TSPTW), we are given a set of locations $N = \{ 0, ..., n-1 \}$.
The salesperson starts from the depot $0$, visit each customer $i \in \{ 1, ..., n-1 \}$ exactly once, and returns to the depot.
The traveling time from $i$ to $j$ is $c_{ij}$.
Each customer $i$ must be visited within time window $[a_i, b_i]$, and the salesperson must wait until $a_i$ if arriving at $i$ before $a_i$.
The objective is to minimize the total travel time (not including the waiting time).

## DP Formulation

Let $U \subseteq N$ be the set of unvisited customers, $i \in N$ be the current location of the salesperson, and $t$ be the current time.
Let $V(U, i, t)$ be the optimal cost to visit customers $U$ and return to the depot starting from $i$ with time $t$.
When customer $j \in U$ is visited next, the problem is reduced to visiting customers $U \setminus \{ j \}$ from location $j$ at time $\max \{ t + c_{ij}, a_j \}$.
When all customers are visited, the salesperson must return to the depot from location $i$. We have the following DP formulation.

$$
\begin{align}
    \text{compute } & V(N \setminus \{ 0 \}, 0, 0) \\ 
    & V(U, i, t) = \begin{cases}
         \min\limits_{j \in U : t + c_{ij} \leq b_j} c_{ij} + V(U \setminus \{ j \}, j, \max \{ t + c_{ij}, a_j \})  & \text{if } U \neq \emptyset \\
         c_{i0} & \text{if } U = \emptyset. \\
    \end{cases}
\end{align}
$$

The earliest arrival time at customer $j$ is $t + c_{ij}$ (assuming the triangle inequality). If $\exists j \in U, t + c_{ij} > b_j$, the state does not lead to a solution.

$$
\begin{align}
    & V(U, i, t) = \infty & \text{if } \exists j \in U, t + c_{ij} > b_j.
\end{align}
$$

When two states $(U, i, t)$ and $(U, i, t')$ have the same set of unvisited customers $U$ and the same location $i$ and $t \leq t'$, $(U, i, t)$ leads to a better solution. Therefore,

$$
\begin{align}
    & V(U, i, t) \leq V(U, i, t') & \text{if } t \leq t'.
\end{align}
$$

The shortest possible travel time to visit customer $j$ is $c^{\text{in}}_j = \min_{k \in N \setminus \{ j \}} c_{kj}$.
Because we need to visit all customers in $U$, the total travel time is at least $\sum_{j \in U} c^{\text{in}}_j$. Furthermore, we need to visit the depot.
Therefore,

$$
    V(U, i, t) \geq \sum_{j \in U \cup \{ 0 \}} c^{\text{in}}_j.
$$

Similarly, we need to depart from each customer in $U$ and the current location $i$. Let $c^{\text{out}}_j = \min_{k \in N \setminus \{ j \}} c_{jk}$. Then,

$$
    V(U, i, t) \geq \sum_{j \in U \cup \{ i \}} c^{\text{out}}_j.
$$

## Install DIDPPy

In [1]:
# Skip this if you already installed it
!pip install didppy

Defaulting to user installation because normal site-packages is not writeable


## Download an Instance Data

In [2]:
# Skip this if you already downloaded it
!wget https://raw.githubusercontent.com/domain-independent-dp/didp-tutorial/main/tsptw_instance.txt

--2024-08-14 14:09:21--  https://raw.githubusercontent.com/domain-independent-dp/didp-tutorial/main/tsptw_instance.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55 [text/plain]
Saving to: ‘tsptw_instance.txt’

tsptw_instance.txt  100%[===================>]      55  --.-KB/s    in 0s      

2024-08-14 14:09:21 (2.23 MB/s) - ‘tsptw_instance.txt’ saved [55/55]



## Read the Instance data

In [3]:
def read_tsptw(filename):
    with open(filename) as f:
        data = f.read().split()

    n = int(data[0])
    data_index = 1
    distance = [[0 for _ in range(n)] for _ in range(n)]
    
    for i in range(n):
        for j in range(n):
            distance[i][j] = int(data[data_index])
            data_index += 1
            
    a = []
    b = []
    
    for i in range(n):
        a.append(int(data[data_index]))
        b.append(int(data[data_index + 1]))
        data_index += 2

    return n, distance, a, b

In [4]:
# n is the number of customers, distance[i][j] is the travel time from i to j,
# and [a[i], b[i]] is the time window for customer i
n, distance, a, b = read_tsptw("tsptw_instance.txt")

In [5]:
print(n)
print(distance)
print(a)
print(b)

4
[[0, 3, 4, 5], [3, 0, 5, 4], [4, 5, 0, 3], [5, 4, 3, 0]]
[0, 5, 0, 8]
[100, 16, 10, 14]


## Modeling

### Import and Set-up Model

In [6]:
import didppy as dp

model = dp.Model(maximize=False, float_cost=False)

### State Variables

$$
\begin{align*}
    \text{compute } & V(N \setminus \{ 0 \}, 0, 0)
\end{align*}
$$

$$
\begin{align*}
    & V(U, i, t) \leq V(U, i, t') & \text{if } t \leq t'
\end{align*}
$$

In [7]:
customer = model.add_object_type(number=n)
u = model.add_set_var(object_type=customer, target=list(range(1, n)))
i = model.add_element_var(object_type=customer, target=0)
t = model.add_int_resource_var(target=0, less_is_better=True)

### Transitions

$$
\begin{align*}
    & V(U, i, j) = \min\limits_{j \in U : t + c_{ij} \leq b_j} c_{ij} + V(U \setminus \{ j \}, j, \max \{ t + c_{ij}, a_j \})  & \text{if } U \neq \emptyset
\end{align*}
$$

In [8]:
c = model.add_int_table(distance)

for j in range(1, n):
    visit = dp.Transition(
        name= "visit {}".format(j),
        cost=c[i,j] + dp.IntExpr.state_cost(),
        effects=[
            (u, u.remove(j)),
            (i, j),
            (t, dp.max(t + c[i,j], a[j])),
        ],
        preconditions=[u.contains(j), t + c[i,j] <= b[j]],
    )
    model.add_transition(visit)

### Base Case

$$
\begin{align*}
    & V(U, i, j) = c_{i0} & \text{if } U = \emptyset
\end{align*}
$$

In [9]:
model.add_base_case([u.is_empty()], cost=c[i,0])

### State Constraints

$$
\begin{align*}
    & V(U, i, t) = \infty & \text{if } \exists j \in U, t + c_{ij} > b_j
\end{align*}
$$

In [10]:
for j in range(1, n):
    model.add_state_constr(
        ~u.contains(j) | (t + c[i,j] <= b[j])
    )

### Dual Bounds

$$
\begin{align*}
    & V(U, i, t) \geq \sum_{j \in U \cup \{ 0 \}} c^{\text{in}}_j \\
    & V(U, i, t) \geq \sum_{j \in U \cup \{ i \}} c^{\text{out}}_j
\end{align*}
$$

In [11]:
cin = model.add_int_table(
    [min(distance[k][j] for k in range(n) if k != j) for j in range(n)]
)
model.add_dual_bound(cin[u] + cin[0])

cout = model.add_int_table(
    [min(distance[j][k] for k in range(n) if k != j) for j in range(n)]
)
model.add_dual_bound(cout[u] + cout[i])

### Solving

In [12]:
solver = dp.CABS(model, time_limit=10, quiet=False)
solution = solver.search()

Solver: CABS from DIDPPy v0.8.0
Searched with beam size: 1, expanded: 3, elapsed time: 0.0001919
New dual bound: 13, expanded: 3, elapsed time: 0.0002266
New primal bound: 16, expanded: 3, elapsed time: 0.0002293
Searched with beam size: 2, expanded: 8, elapsed time: 0.0002722
New dual bound: 14, expanded: 8, elapsed time: 0.0002747
New primal bound: 14, expanded: 8, elapsed time: 0.0002803


In [13]:
print("Solution:")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))
print("Elapsed time: {:.4f} seconds".format(solution.time))

if solution.is_optimal:
    print("The optimality is proved")
else:
    print("Best bound: {}".format(solution.best_bound))

Solution:
visit 2
visit 3
visit 1

Cost: 14
Elapsed time: 0.0003 seconds
The optimality is proved
